# Create an ontology

Using the [OwlReady2](https://owlready2.readthedocs.io/en/latest/index.html) package.

Import the library and define the local `ontologies` folder. If an URL is given, first searches for a local copy of the OWL file and, if not found, tries to download it from the Internet.

In [1]:
from owlready2 import *
import types

global my_family_onto 
global audioset_curated_hash
onto_path.append("/notebooks/ontologies")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


### Create and load ontologies

Create an empty ontology and load the Pizza ontology from the Internet (for example purpose)

In [2]:
my_family_onto = get_ontology("https://w3id.org/myonto")

stevensr_family_onto = get_ontology("http://www.cs.man.ac.uk/~stevensr/ontology/family.rdf.owl").load()

### Example to generate properties with domain and ranges

See [OwlReady2 documentation](https://owlready2.readthedocs.io/en/latest/index.html) for:
* [Dynamic Classes](https://owlready2.readthedocs.io/en/latest/class.html#creating-classes-dynamically)
* [Add annotations to a Class](https://owlready2.readthedocs.io/en/latest/annotations.html?highlight=comment#adding-an-annotation): `comment`, `isDefinedBy`, `label`, `seeAlso`, `backwardCompatibleWith`, `deprecated`, `incompatibleWith`, `priorVersion`, `versionInfo`
* [Properties](https://owlready2.readthedocs.io/en/latest/properties.html): FunctionalProperty, InverseFunctionalProperty, TransitiveProperty, SymmetricProperty, AsymmetricProperty, ReflexiveProperty, IrreflexiveProperty


In [3]:
with my_family_onto:
    class Person(Thing):
        label = "A person"
        comment = "Any person"
        pass
    class Man(Person):
        pass
    class Woman(Person):
        pass
    class has_parent(ObjectProperty):
        domain    = [Person]
        range     = [Person]
    class is_parent_of(ObjectProperty):
        domain    = [Person]
        range     = [Person]
        inverse_property = has_parent
    class has_father(has_parent):
        domain    = [Person]
        range     = [Man]
    class has_mother(has_parent):
        domain    = [Person]
        range     = [Woman]
    class family_name(DataProperty, FunctionalProperty):
        domain    = [Person]
        range     = [str]
    class Dad(Man):
        equivalent_to = [Man & is_parent_of.some(Person)]
    class Mom(Woman):
        equivalent_to = [Woman & is_parent_of.some(Person)]

### Add instance

In [4]:
with my_family_onto:
    Person("Jean", family_name="Dupont")

### Add metadata to the ontology

In [5]:
my_family_onto.metadata.comment.append("OWL Ontology to describe a family and its relations.")

### Save the ontology file

Ontology files saved in the `ontologies` folder. 

2 formats available, defined in the papermill parameters (at the start of the notebook, or in the `papermill-config.json` file):
* `rdfxml`
* `ntriples`

In [6]:
my_family_onto.save(file = "ontologies/my_family_ontology.rdf", format = "rdfxml")

# Explore the ontology

**With OwlReady2**, e.g. list the ontology classes and properties.

In [7]:
# Get a class IRI:
print(my_family_onto.Person.iri)
# List all classes:
print(list(my_family_onto.classes()))
# List object properties:
print(list(my_family_onto.object_properties()))
# List a class instances:
for i in my_family_onto.Person.instances(): print(i)

https://w3id.org/myonto#Person
[myonto.Person, myonto.Man, myonto.Woman, myonto.Dad, myonto.Mom]
[myonto.has_parent, myonto.is_parent_of, myonto.has_father, myonto.has_mother]
myonto.Jean


### Use Ontospy to analyze the ontology

Load the ontology file with `ontospy`, then:
* print top classes and the class tree
* print instances of a class

In [8]:
import ontospy
my_family_spy = ontospy.Ontospy("ontologies/my_family_ontology.rdf", verbose=True)

Reading: <ontologies/my_family_ontology.rdf>
.. trying rdf serialization: <xml>
..... success!
----------
Loaded 57 triples.
----------
RDF sources loaded successfully: 1 of 1.
..... 'ontologies/my_family_ontology.rdf'
----------
Scanning entities...
----------
Ontologies.........: 1
Classes............: 5
Properties.........: 5
..annotation.......: 0
..datatype.........: 1
..object...........: 4
Concepts (SKOS)....: 0
Shapes (SHACL).....: 0
----------


In [9]:
my_family_spy.printClassTree()
my_family_spy.toplayer_classes

myonto:Person
----myonto:Man
--------myonto:Dad
----myonto:Woman
--------myonto:Mom


[<Class *https://w3id.org/myonto#Person*>]

In [10]:
# Print instances of Sigh class
my_family_spy.get_class('Person')[0]
for instance in my_family_spy.get_class('Person')[0].instances:
        print(instance.uri, instance.qname)
        instance.printTriples()

https://w3id.org/myonto#Jean myonto:Jean
https://w3id.org/myonto#Jean
=> http://www.w3.org/1999/02/22-rdf-syntax-ns#type
.... http://www.w3.org/2002/07/owl#NamedIndividual
=> http://www.w3.org/1999/02/22-rdf-syntax-ns#type
.... https://w3id.org/myonto#Person
=> https://w3id.org/myonto#family_name
.... Dupont



### Visualize with Ontospy docs

Experimental, it is recommended to generate the documentation from the commandline (cf. `README.md` file) 

In [11]:
# from ontospy.ontodocs.viz.viz_html_single import *

# v = HTMLVisualizer(audioset_spy) # => instantiate the visualization object
# v.build("/notebooks/docs") # => render visualization. You can pass an 'output_path' parameter too
# v.preview() # => open in browser

### Visualize with WebVOWL

Use the URL to the ontology file:

[http://www.visualdataweb.de/webvowl/#iri=https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN3140_SemanticWeb/master/ontologies/my_family_ontology.rdf](http://www.visualdataweb.de/webvowl/#iri=https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN3140_SemanticWeb/master/ontologies/my_family_ontology.rdf)

### Visualize as graph using networkx

Use `rdflib` and `networkx` to load the data in the graph and display it (not working with the ontology size, to be improved.

In [12]:
# import rdflib
# from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
# import networkx as nx
# import matplotlib.pyplot as plt

# g = rdflib.Graph()
# result = g.parse('ontologies/my_family_ontology.rdf', format='xml')

# G = rdflib_to_networkx_multidigraph(result)

# # Plot Networkx instance of RDF Graph
# pos = nx.spring_layout(G, scale=3)
# edge_labels = nx.get_edge_attributes(G, 'r')
# nx.draw_networkx_edge_labels(G, pos, labels=edge_labels)
# nx.draw(G, with_labels=True)